In [1]:
import numpy as np
import cv2
from scipy.spatial import distance

cap = cv2.VideoCapture('multiant.mp4')

#cv2.namedWindow('frame',cv2.WINDOW_NORMAL)

fgbg = cv2.createBackgroundSubtractorMOG2()
kernel = np.ones((2,2),np.uint8)
kerneld = np.ones((4,4),np.uint8)

while(cap.isOpened()):
    
    ret, colorim = cap.read()
    ro,co,ch=colorim.shape

    if not ret:
        break

       
    #frame = cv2.bilateralFilter(colorim,4,15,15)
    frame = cv2.cvtColor(colorim, cv2.COLOR_BGR2GRAY)

    frame = fgbg.apply(frame)
    #frame = cv2.blur(frame,(5,5))
    frame = cv2.erode(frame,kernel,iterations = 2)
    frame = cv2.dilate(frame,kerneld,iterations = 5)
    #more dialation gives bigger representations of ant
    
    ret,frame = cv2.threshold(frame,127,255,0)
 
    contours, hierarchy = cv2.findContours(frame,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    for c in contours:
    # compute the center of the contour
        M = cv2.moments(c)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        # draw the contour and center of the shape on the image
        #cv2.drawContours(colorim, [c], -1, (0, 255, 0), 2)
        colorim=cv2.circle(colorim, (cX, cY), 18, (255, 0, 255), 2)
        
        
        cv2.line(colorim,(cX,0),(cX,1000),(0,0,255),1)
        cv2.line(colorim,(0,cY),(2000,cY),(0,0,255),1)
        cv2.line(colorim,(int(co/2),int(ro/2)),(cX,cY),(0,255,0),2)
        dist = distance.euclidean((int(co/2),int(ro/2)), (cX,cY))
        #colorim=cv2.putText(colorim, "ant", (cX - 20, cY - 20),
            #cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        colorim=cv2.putText(colorim, str(dist), (cX - 20, cY - 20),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        
         

    colorim=cv2.circle(colorim, (int(co/2),int(ro/2)), 18, (255, 255, 255), -1)
    cv2.imshow('frame',colorim)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()